# Investment Returns Analysis with Polars

This notebook demonstrates using Polars for investment portfolio analysis, including:
- Common Polars DataFrame operations
- Efficient data transformations
- Performance comparisons with a VOO benchmark
- XIRR calculations for money-weighted returns

In [10]:
# Core libraries
import polars as pl
import polars.selectors as cs
import numpy as np
from pathlib import Path
from datetime import datetime, date
import warnings
warnings.filterwarnings('ignore')

# For XIRR calculations
import numpy_financial as npf

# For data fetching using Nasdaq Data Link
import os
import nasdaqdatalink
from typing import Dict, List, Tuple
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# For visualization (still need matplotlib for plots)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# Configure Polars display options
pl.Config.set_tbl_rows(10)
pl.Config.set_fmt_str_lengths(50)

# Configure Nasdaq Data Link
api_key = os.getenv('NASDAQ_DATA_LINK_API_KEY')
if api_key:
    nasdaqdatalink.ApiConfig.api_key = api_key
    # Enable retries for robustness
    nasdaqdatalink.ApiConfig.use_retries = True
    nasdaqdatalink.ApiConfig.number_of_retries = 3
    nasdaqdatalink.ApiConfig.retry_status_codes = [429, 500, 501, 502, 503, 504]

print(f"Polars version: {pl.__version__}")
print(f"Nasdaq Data Link configured: {'Yes' if api_key else 'No - set NASDAQ_DATA_LINK_API_KEY in .env file'}")
print("All libraries imported successfully")

Polars version: 1.32.3
Nasdaq Data Link configured: Yes
All libraries imported successfully


In [11]:
def load_transactions():
    """Load all transaction CSV files and combine them using Polars"""
    transactions_dir = Path('/Users/bhargav/Git/investments/transactions')
    all_transactions = []
    
    # Define schema to ensure correct data types
    schema_overrides = {
        "Price USD": pl.Float64,
        "Quantity": pl.Float64,
        "Amount USD": pl.Float64
    }
    
    for csv_file in sorted(transactions_dir.glob('*.csv')):
        # Read CSV with Polars, specifying schema overrides
        df = pl.read_csv(csv_file, schema_overrides=schema_overrides)
        # Clean column names (remove BOM if present)
        df = df.rename({col: col.replace('﻿', '') for col in df.columns})
        all_transactions.append(df)
        print(f"Loaded {csv_file.name}: {len(df)} transactions")
    
    # Combine all years using Polars concat
    transactions = pl.concat(all_transactions, how="vertical")
    
    # Convert date column and sort (fixed date format)
    transactions = (
        transactions
        .with_columns(
            pl.col("Trade Date").str.to_date(format="%m/%d/%Y")  # Fixed date format
        )
        .sort("Trade Date")
    )
    
    return transactions

# Load the data
transactions = load_transactions()
print(f"\nTotal transactions loaded: {len(transactions)}")

Loaded 2018.csv: 42 transactions
Loaded 2019.csv: 42 transactions
Loaded 2020.csv: 195 transactions
Loaded 2021.csv: 218 transactions
Loaded 2022.csv: 289 transactions
Loaded 2023.csv: 257 transactions
Loaded 2024.csv: 382 transactions
Loaded 2025.csv: 251 transactions

Total transactions loaded: 1676


In [12]:
def consolidate_transaction_types(df):
    """
    Consolidate duplicate and similar transaction types using Polars
    """
    # Create mapping for consolidation
    type_mapping = {
        # Deposits - consolidate variations
        'DEPOSIT': 'Deposit',
        'Deposit': 'Deposit',
        
        # Splits - consolidate variations
        'STK SPLT': 'Split',
        'SPLT': 'Split',
        'Split': 'Split',
        
        # Dividends - keep as is
        'Dividend': 'Dividend',
        'DBS': 'Dividend',
        'DBT': 'Dividend',
        
        # Reinvestments
        'Reinvest': 'Reinvest',
        
        # Buys and Sells
        'Buy': 'Buy',
        'Sell': 'Sell',
        'LIQ': 'Sell',      # Liquidation
        
        # Interest
        'Interest': 'Interest',
        
        # Distributions and capital gains
        'Distribution': 'Distribution',
        'CAP': 'Capital Gain',
        
        # Tax-related
        'WHT': 'Tax Withheld',     # Withholding tax
        'FWT': 'Tax Withheld',     # Foreign withholding tax
        
        # Fees
        'ADR': 'Fee',          # ADR fee
        'MER': 'Fee',    # Management expense ratio
        
        # Corporate actions
        'WDL': 'Withdrawal',       # Withdrawal
        'BNK': 'Bank Transfer',    # Bank transfer
        'CIL': 'Corporate Action',     # Cash in lieu
        'Exchange': 'Corporate Action',     # Currency exchange or security exchange
    }
    
    # Apply the mapping using Polars replace and then uppercase
    df = df.with_columns(
        pl.col("Type")
        .replace(type_mapping, default=pl.col("Type"))
        .str.strip_chars()
        .str.to_uppercase()
    )

    return df

transactions = consolidate_transaction_types(transactions)
print(transactions.head())

shape: (5, 7)
┌────────────┬──────────┬────────┬───────────────┬───────────┬──────────┬────────────┐
│ Trade Date ┆ Type     ┆ Ticker ┆ Security Type ┆ Price USD ┆ Quantity ┆ Amount USD │
│ ---        ┆ ---      ┆ ---    ┆ ---           ┆ ---       ┆ ---      ┆ ---        │
│ date       ┆ str      ┆ str    ┆ str           ┆ f64       ┆ f64      ┆ f64        │
╞════════════╪══════════╪════════╪═══════════════╪═══════════╪══════════╪════════════╡
│ 2018-07-19 ┆ BUY      ┆ SCHF   ┆ ETF           ┆ 33.25     ┆ 74.0     ┆ -2460.5    │
│ 2018-07-19 ┆ BUY      ┆ SCHX   ┆ ETF           ┆ 67.0      ┆ 74.0     ┆ -4958.0    │
│ 2018-07-24 ┆ BUY      ┆ SCHA   ┆ ETF           ┆ 75.0      ┆ 25.0     ┆ -1875.0    │
│ 2018-09-12 ┆ BUY      ┆ FTEC   ┆ ETF           ┆ 59.42     ┆ 40.0     ┆ -2376.8    │
│ 2018-09-27 ┆ REINVEST ┆ FTEC   ┆ ETF           ┆ 60.19     ┆ 0.0937   ┆ -5.64      │
└────────────┴──────────┴────────┴───────────────┴───────────┴──────────┴────────────┘


In [13]:
print("Transaction consolidated:")
print(
    transactions
    .group_by("Type")
    .agg(pl.count().alias("count"))
    .sort("count", descending=True)
)

Transaction consolidated:
shape: (14, 2)
┌──────────────────┬───────┐
│ Type             ┆ count │
│ ---              ┆ ---   │
│ str              ┆ u32   │
╞══════════════════╪═══════╡
│ DIVIDEND         ┆ 619   │
│ REINVEST         ┆ 415   │
│ BUY              ┆ 398   │
│ WITHDRAWAL       ┆ 68    │
│ INTEREST         ┆ 57    │
│ …                ┆ …     │
│ BANK TRANSFER    ┆ 13    │
│ SPLIT            ┆ 10    │
│ CAPITAL GAIN     ┆ 10    │
│ CORPORATE ACTION ┆ 4     │
│ DISTRIBUTION     ┆ 1     │
└──────────────────┴───────┘


In [14]:
# More advanced Polars operations
print("ADVANCED POLARS OPERATIONS")
print("=" * 60)

# 1. Date operations
print("\n1. DATE OPERATIONS - Extract date components:")
date_analysis = (
    transactions
    .with_columns([
        pl.col("Trade Date").dt.year().alias("year"),
        pl.col("Trade Date").dt.month().alias("month"),
        pl.col("Trade Date").dt.quarter().alias("quarter"),
        pl.col("Trade Date").dt.weekday().alias("weekday")
    ])
    .select(["Trade Date", "year", "month", "quarter", "weekday"])
    .head(10)
)
print(date_analysis)

# 2. Pivot table
print("\n2. PIVOT TABLE - Transactions by year and type:")
pivot_result = (
    transactions
    .with_columns(
        pl.col("Trade Date").dt.year().alias("Year")
    )
    .pivot(
        values="Amount USD",
        index="Year", 
        columns="Type",
        aggregate_function="sum"
    )
    .fill_null(0)
)
print(pivot_result)

# 3. Conditional operations with when/then/otherwise
print("\n3. CONDITIONAL LOGIC - Categorize transaction sizes:")
categorized = (
    transactions
    .with_columns(
        pl.when(pl.col("Amount USD") < -10000)
        .then(pl.lit("Large Purchase"))
        .when(pl.col("Amount USD") < -1000)
        .then(pl.lit("Medium Purchase"))
        .when(pl.col("Amount USD") < 0)
        .then(pl.lit("Small Purchase"))
        .when(pl.col("Amount USD") > 1000)
        .then(pl.lit("Large Income"))
        .when(pl.col("Amount USD") > 0)
        .then(pl.lit("Small Income"))
        .otherwise(pl.lit("Zero"))
        .alias("transaction_category")
    )
    .group_by("transaction_category")
    .agg([
        pl.count().alias("count"),
        pl.col("Amount USD").mean().alias("avg_amount")
    ])
)
print(categorized)

ADVANCED POLARS OPERATIONS

1. DATE OPERATIONS - Extract date components:
shape: (10, 5)
┌────────────┬──────┬───────┬─────────┬─────────┐
│ Trade Date ┆ year ┆ month ┆ quarter ┆ weekday │
│ ---        ┆ ---  ┆ ---   ┆ ---     ┆ ---     │
│ date       ┆ i32  ┆ i8    ┆ i8      ┆ i8      │
╞════════════╪══════╪═══════╪═════════╪═════════╡
│ 2018-07-19 ┆ 2018 ┆ 7     ┆ 3       ┆ 4       │
│ 2018-07-19 ┆ 2018 ┆ 7     ┆ 3       ┆ 4       │
│ 2018-07-24 ┆ 2018 ┆ 7     ┆ 3       ┆ 2       │
│ 2018-09-12 ┆ 2018 ┆ 9     ┆ 3       ┆ 3       │
│ 2018-09-27 ┆ 2018 ┆ 9     ┆ 3       ┆ 4       │
│ 2018-10-01 ┆ 2018 ┆ 10    ┆ 4       ┆ 1       │
│ 2018-10-01 ┆ 2018 ┆ 10    ┆ 4       ┆ 1       │
│ 2018-10-02 ┆ 2018 ┆ 10    ┆ 4       ┆ 2       │
│ 2018-10-08 ┆ 2018 ┆ 10    ┆ 4       ┆ 1       │
│ 2018-10-09 ┆ 2018 ┆ 10    ┆ 4       ┆ 2       │
└────────────┴──────┴───────┴─────────┴─────────┘

2. PIVOT TABLE - Transactions by year and type:
shape: (8, 15)
┌──────┬────────────┬───────────┬──────────┬───

In [15]:
def filter_equity_transactions(df, MONEY_MARKET_FUNDS = ['VMFXX', 'QACDS', 'SPAXX', 'FDRXX', 'SWVXX', 'VMMXX']):
    """Filter for equity/ETF transactions only using Polars"""
    # Chain operations for efficient filtering
    df_filtered = (
        df
        # Remove money market funds
        .filter(~pl.col("Ticker").is_in(MONEY_MARKET_FUNDS))
        # Keep only relevant transaction types
        .filter(pl.col("Type").is_in(['BUY', 'SELL', 'DIVIDEND', 'REINVEST', 'CAPITAL GAIN']))
        # Remove rows without tickers
        .filter(pl.col("Ticker").is_not_null())
    )
    
    return df_filtered
equity_transactions = filter_equity_transactions(transactions)
print(f"Filtered to {len(equity_transactions)} equity transactions")

print(f"\nUnique tickers ({equity_transactions['Ticker'].n_unique()}):")
unique_tickers = (
    equity_transactions
    .select("Ticker")
    .unique()
    .sort("Ticker")
    .get_column("Ticker")
    .to_list()
)
print(sorted(unique_tickers))

Filtered to 1284 equity transactions

Unique tickers (83):
[' NOV 24 PUT 517.50', 'AAPL', 'ABNB', 'ADSK', 'ADYEY', 'AMD', 'AMZN', 'ANET', 'APPN', 'ASML', 'ATVI', 'AVB', 'BABA', 'BRKB', 'BTCO', 'CFLT', 'CHGG', 'CRM', 'CRWD', 'DIS', 'DXCM', 'ETSY', 'FB', 'FBSOX', 'FCOM', 'FHLC', 'FSLY', 'FTEC', 'FTNT', 'FUBO', 'FVRR', 'GS', 'HUBS', 'IDXX', 'IYW', 'JATAX', 'JD', 'LKNCY', 'LMND', 'MA', 'MELI', 'META', 'MKC', 'MSFT', 'MTN', 'NET', 'NFLX', 'NICE', 'NTDOY', 'NVDA', 'OKTA', 'PAYC', 'PINS', 'POTX', 'PYPL', 'QQQ', 'ROKU', 'SCHA', 'SCHF', 'SCHX', 'SCHZ', 'SHOP', 'SNBR', 'SNPS', 'TDOC', 'TEAM', 'TSLA', 'TTD', 'TWLO', 'TXG', 'UAL', 'UBER', 'UPST', 'VDC', 'VGT', 'VMSXX', 'VOO', 'VOX', 'VTI', 'WEX', 'ZBRA', 'ZM', 'ZNGA']


In [16]:
class NasdaqDataFetcher:
    """Fetch historical data from Nasdaq Data Link using the official library - returns Polars DataFrames"""
    
    def __init__(self, api_key=None):
        self.api_key = api_key or os.getenv('NASDAQ_DATA_LINK_API_KEY')
        if self.api_key:
            nasdaqdatalink.ApiConfig.api_key = self.api_key
            # Configure robustness features
            nasdaqdatalink.ApiConfig.use_retries = True
            nasdaqdatalink.ApiConfig.number_of_retries = 3
            nasdaqdatalink.ApiConfig.retry_status_codes = [429, 500, 501, 502, 503, 504]
        else:
            print("WARNING: No API key found. Set NASDAQ_DATA_LINK_API_KEY in .env file")
            print("You can get a free API key at: https://data.nasdaq.com/sign-up")
        self.cache = {}
    
    def get_price_history(self, ticker: str, start_date: str, end_date: str) -> pl.DataFrame:
        """Get daily price history for a ticker including adjusted close and dividends using SHARADAR/SEP"""
        # Check cache first
        cache_key = f"{ticker}_{start_date}_{end_date}"
        if cache_key in self.cache:
            return self.cache[cache_key]
        
        if not self.api_key:
            return None
        
        try:
            # Use the nasdaq-data-link library to fetch from SHARADAR/SEP table
            # This provides adjusted close prices and dividend information
            data = nasdaqdatalink.get_table(
                'SHARADAR/SEP',
                ticker=ticker,
                date={'gte': start_date, 'lte': end_date},
                qopts={'columns': ['ticker', 'date', 'closeadj', 'divamt']},
                paginate=True  # Automatically handle pagination for large datasets
            )
            
            if data is not None and not data.empty:
                # Convert pandas DataFrame to Polars for consistency
                df = pl.from_pandas(data)
                
                # Ensure date column is properly typed
                if 'date' in df.columns:
                    df = df.with_columns(
                        pl.col('date').cast(pl.Date)
                    ).sort('date')
                
                self.cache[cache_key] = df
                return df
            else:
                print(f"No data found for {ticker} in the specified date range")
                return pl.DataFrame()
                
        except nasdaqdatalink.NotFoundError:
            print(f"Ticker {ticker} not found in SHARADAR/SEP database")
            return pl.DataFrame()
            
        except nasdaqdatalink.LimitExceededError:
            print(f"API limit exceeded. Please wait.")
            return pl.DataFrame()
            
        except nasdaqdatalink.AuthenticationError:
            print(f"Authentication failed. Please check your API key.")
            return pl.DataFrame()
            
        except Exception as e:
            print(f"Error fetching {ticker}: {e}")
            return pl.DataFrame()
    
    def get_fund_prices(self, ticker: str, start_date: str, end_date: str) -> pl.DataFrame:
        """Get fund prices from SHARADAR/SFP for ETFs and mutual funds"""
        
        cache_key = f"fund_{ticker}_{start_date}_{end_date}"
        if cache_key in self.cache:
            return self.cache[cache_key]
        
        if not self.api_key:
            return self.get_price_history(ticker, start_date, end_date)  # Use dummy data
        
        try:
            # Try SHARADAR/SFP for funds (ETFs, mutual funds)
            data = nasdaqdatalink.get_table(
                'SHARADAR/SFP',
                ticker=ticker,
                date={'gte': start_date, 'lte': end_date},
                qopts={'columns': ['ticker', 'date', 'closeadj', 'divamt']},
                paginate=True
            )
            
            if data is not None and not data.empty:
                df = pl.from_pandas(data)
                if 'date' in df.columns:
                    df = df.with_columns(
                        pl.col('date').cast(pl.Date)
                    ).sort('date')
                self.cache[cache_key] = df
                return df
            else:
                return pl.DataFrame()
                
        except Exception:
            # If not found in SFP, fallback to SEP
            return self.get_price_history(ticker, start_date, end_date)
    
    def get_ticker_info(self, ticker: str) -> dict:
        """Get ticker metadata from SHARADAR/TICKERS table"""
        
        if not self.api_key:
            return {}
        
        try:
            data = nasdaqdatalink.get_table(
                'SHARADAR/TICKERS',
                ticker=ticker,
                qopts={'columns': ['ticker', 'name', 'exchange', 'isdelisted', 'category', 'sector', 'industry']}
            )
            
            if data is not None and not data.empty:
                return data.iloc[0].to_dict()
            return {}
            
        except Exception as e:
            print(f"Error fetching ticker info for {ticker}: {e}")
            return {}

# Initialize the fetcher with built-in robustness
fetcher = NasdaqDataFetcher()
print("Nasdaq Data Fetcher initialized with official library")
print("API Key status:", "Set" if fetcher.api_key else "Not set (using dummy data)")
print("\nBuilt-in robustness features:")
print("  - Automatic retries: 3 attempts")
print("  - Handles rate limiting (429 errors)")
print("  - Handles server errors (500-504)")
print("  - Automatic pagination for large datasets")
print("  - Caching to minimize API calls")

Nasdaq Data Fetcher initialized with official library
API Key status: Set

Built-in robustness features:
  - Automatic retries: 3 attempts
  - Handles rate limiting (429 errors)
  - Handles server errors (500-504)
  - Automatic pagination for large datasets
  - Caching to minimize API calls


## Step 2: Filter Relevant Transactions

In [ ]:
class Portfolio:
    """Track portfolio holdings and calculate values over time using Polars"""
    
    def __init__(self):
        self.holdings = {}  # ticker -> shares
        self.cash_flows = []  # List of (date, amount) for XIRR
        self.history = []  # Daily portfolio values
    
    def process_transaction(self, date, txn_type, ticker, quantity, amount):
        """Process a single transaction"""
        if ticker not in self.holdings:
            self.holdings[ticker] = 0
        
        if txn_type in ['BUY']:
            # Money leaves account (negative cash flow)
            self.holdings[ticker] += abs(quantity)
            self.cash_flows.append((date, -abs(amount)))
            
        elif txn_type in ['REINVEST']:
            # Dividend reinvested - no net cash flow for XIRR
            # But we do increase share count
            self.holdings[ticker] += abs(quantity)
            
        elif txn_type == 'SELL':
            # Money enters account (positive cash flow)
            self.holdings[ticker] -= abs(quantity)
            self.cash_flows.append((date, abs(amount)))
            
        elif txn_type == 'DIVIDEND' and quantity == 0:
            # Cash dividend (not reinvested)
            self.cash_flows.append((date, abs(amount)))
            
        elif txn_type == 'CAPITAL GAIN':
            # Capital gains distribution (usually mutual funds)
            # If reinvested, no cash flow
            if quantity != 0:
                self.holdings[ticker] += abs(quantity)
    
    def calculate_value(self, date, price_data):
        """Calculate total portfolio value on a given date using Polars DataFrames"""
        total_value = 0
        for ticker, shares in self.holdings.items():
            if shares > 0 and ticker in price_data:
                # Get most recent price up to this date
                ticker_prices = price_data[ticker]
                prices_before = ticker_prices.filter(pl.col('date') <= date)
                if len(prices_before) > 0:
                    # Get the last price using Polars
                    price = prices_before['closeadj'][-1]
                    total_value += shares * price
        return total_value
    
    def calculate_xirr(self, end_date, end_value):
        """Calculate XIRR including final portfolio value"""
        if len(self.cash_flows) == 0:
            return 0
        
        # Add final portfolio value as positive cash flow
        all_flows = self.cash_flows + [(end_date, end_value)]
        
        # Separate dates and amounts for numpy_financial
        dates = [cf[0] for cf in all_flows]
        amounts = [cf[1] for cf in all_flows]
        
        # Convert dates to datetime if they're not already
        dates_dt = []
        for d in dates:
            if isinstance(d, datetime):
                dates_dt.append(d)
            else:
                dates_dt.append(datetime.combine(d, datetime.min.time()))
        
        try:
            xirr = npf.xirr(amounts, dates_dt)
            return xirr * 100 if xirr is not None else None  # Convert to percentage
        except:
            # XIRR calculation can fail if returns are extreme
            return None

print("Portfolio tracking class defined with Polars support")

## Step 3: Nasdaq Data Link Setup

In [ ]:
# Get date range and tickers using Polars operations
date_stats = equity_transactions.select([
    pl.col('Trade Date').min().alias('start_date'),
    pl.col('Trade Date').max().alias('end_date')
]).row(0)

start_date = date_stats[0].strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')

tickers = equity_transactions['Ticker'].unique().to_list()

print(f"Fetching data for {len(tickers)} tickers")
print(f"Date range: {start_date} to {end_date}")

# Fetch price data
price_data = {}
failed_tickers = []

# Common ETF list for trying SFP table first
COMMON_ETFS = ['VOO', 'VTI', 'QQQ', 'SCHX', 'SCHF', 'SCHA', 'SCHZ', 'VGT', 'VDC', 'VOX', 
               'FTEC', 'FHLC', 'FCOM', 'IYW', 'FBSOX', 'BTCO', 'POTX']

# Always fetch VOO for benchmark
print("\nFetching VOO benchmark data...")
voo_prices = fetcher.get_fund_prices('VOO', start_date, end_date)
if voo_prices.is_empty():
    # Try SEP if SFP fails
    voo_prices = fetcher.get_price_history('VOO', start_date, end_date)
if not voo_prices.is_empty():
    price_data['VOO'] = voo_prices
    print(f"  ✓ VOO: {len(voo_prices)} days of data")
else:
    print(f"  ✗ VOO: No data available")

# Fetch data for holdings (limit to first 20 for demo, remove limit for full analysis)
print(f"\nFetching price data for portfolio holdings...")
ticker_limit = min(20, len(tickers))  # Adjust or remove this limit as needed

for i, ticker in enumerate(tickers[:ticker_limit]):
    if ticker and ' ' not in ticker and ticker != 'VOO':  # Skip invalid tickers and VOO (already fetched)
        print(f"  [{i+1}/{ticker_limit}] {ticker}...", end=" ")
        
        # Try fund table first for known ETFs
        if ticker in COMMON_ETFS:
            prices = fetcher.get_fund_prices(ticker, start_date, end_date)
            if prices.is_empty():
                # Fallback to equity table
                prices = fetcher.get_price_history(ticker, start_date, end_date)
        else:
            # Try equity table first for stocks
            prices = fetcher.get_price_history(ticker, start_date, end_date)
            if prices.is_empty():
                # Fallback to fund table
                prices = fetcher.get_fund_prices(ticker, start_date, end_date)
        
        if not prices.is_empty():
            price_data[ticker] = prices
            print(f"✓ ({len(prices)} days)")
        else:
            failed_tickers.append(ticker)
            print(f"✗ (no data)")

print(f"\n" + "="*60)
print(f"Successfully fetched data for {len(price_data)} tickers")
if failed_tickers:
    print(f"Failed to fetch data for {len(failed_tickers)} tickers: {', '.join(failed_tickers[:10])}")
    if len(failed_tickers) > 10:
        print(f"  ... and {len(failed_tickers) - 10} more")

# Show sample of fetched data
if price_data:
    sample_ticker = list(price_data.keys())[0]
    print(f"\nSample data for {sample_ticker}:")
    print(price_data[sample_ticker].head())

## Step 4: Portfolio Tracking Functions

In [ ]:
# Create actual portfolio
actual_portfolio = Portfolio()

# Process all transactions - convert Polars DataFrame to records for iteration
for row in equity_transactions.iter_rows(named=True):
    actual_portfolio.process_transaction(
        date=row['Trade Date'],
        txn_type=row['Type'],
        ticker=row['Ticker'],
        quantity=row.get('Quantity', 0) or 0,
        amount=row.get('Amount USD', 0) or 0
    )

# Calculate final portfolio value
final_date = equity_transactions['Trade Date'].max()
final_value = actual_portfolio.calculate_value(final_date, price_data)

# Calculate XIRR
actual_xirr = actual_portfolio.calculate_xirr(final_date, final_value)

print(f"Actual Portfolio Summary:")
print(f"  Final Value: ${final_value:,.2f}")
print(f"  Number of transactions: {len(equity_transactions)}")
print(f"  Number of cash flows: {len(actual_portfolio.cash_flows)}")
if actual_xirr:
    print(f"  XIRR (Annualized Return): {actual_xirr:.2f}%")
else:
    print(f"  XIRR: Could not calculate")

In [ ]:
# Demonstrate Polars join operations
print("POLARS JOIN OPERATIONS")
print("=" * 60)

# Create a summary of holdings
holdings_summary = (
    equity_transactions
    .filter(pl.col('Type').is_in(['BUY', 'SELL']))
    .with_columns(
        pl.when(pl.col('Type') == 'SELL')
        .then(-pl.col('Quantity'))
        .otherwise(pl.col('Quantity'))
        .alias('adjusted_quantity')
    )
    .group_by('Ticker')
    .agg([
        pl.col('adjusted_quantity').sum().alias('net_shares'),
        pl.col('Amount USD').filter(pl.col('Type') == 'BUY').sum().abs().alias('total_invested'),
        pl.count().alias('transaction_count')
    ])
    .filter(pl.col('net_shares') > 0)
)

print("\n1. Current Holdings Summary:")
print(holdings_summary.head(10))

# If we have price data, join with current prices
if price_data:
    # Get latest prices for each ticker
    latest_prices = []
    for ticker, prices_df in price_data.items():
        if not prices_df.is_empty():
            latest_price = pl.DataFrame({
                'Ticker': [ticker],
                'current_price': [prices_df['closeadj'][-1]]
            })
            latest_prices.append(latest_price)
    
    if latest_prices:
        prices_df = pl.concat(latest_prices)
        
        # Join holdings with prices
        portfolio_values = (
            holdings_summary
            .join(prices_df, on='Ticker', how='left')
            .with_columns(
                (pl.col('net_shares') * pl.col('current_price')).alias('current_value')
            )
            .with_columns(
                ((pl.col('current_value') - pl.col('total_invested')) / pl.col('total_invested') * 100)
                .alias('return_pct')
            )
            .sort('current_value', descending=True)
        )
        
        print("\n2. Portfolio Values with Returns:")
        print(portfolio_values.head(10))

def simulate_voo_portfolio(transactions_df, voo_prices):
    """Simulate portfolio if all Buy transactions went to VOO instead"""
    voo_portfolio = Portfolio()
    
    # Get all buy transactions using Polars
    buy_transactions = transactions_df.filter(pl.col('Type') == 'BUY').sort('Trade Date')
    
    for row in buy_transactions.iter_rows(named=True):
        date = row['Trade Date']
        amount = abs(row['Amount USD'])
        
        # Find VOO price on that date using Polars
        voo_on_date = voo_prices.filter(pl.col('date') <= date)
        if len(voo_on_date) > 0:
            voo_price = voo_on_date['closeadj'][-1]
            shares = amount / voo_price
            
            # Simulate buying VOO
            voo_portfolio.process_transaction(
                date=date,
                txn_type='BUY',
                ticker='VOO',
                quantity=shares,
                amount=amount
            )
    
    # Handle dividend reinvestment for VOO
    # Get all dividend payments for VOO during holding period
    if len(voo_portfolio.holdings) > 0 and 'VOO' in voo_portfolio.holdings:
        voo_divs = voo_prices.filter(pl.col('divamt') > 0)
        min_date = buy_transactions['Trade Date'].min()
        
        for row in voo_divs.iter_rows(named=True):
            div_date = row['date']
            # Only process if we held shares on this date
            if div_date >= min_date:
                shares_held = voo_portfolio.holdings.get('VOO', 0)
                if shares_held > 0:
                    div_amount = shares_held * row['divamt']
                    new_shares = div_amount / row['closeadj']
                    # Reinvest dividend
                    voo_portfolio.process_transaction(
                        date=div_date,
                        txn_type='REINVEST',
                        ticker='VOO',
                        quantity=new_shares,
                        amount=div_amount
                    )
    
    return voo_portfolio

# Simulate VOO portfolio
if 'VOO' in price_data:
    voo_portfolio = simulate_voo_portfolio(equity_transactions, price_data['VOO'])
    voo_final_value = voo_portfolio.calculate_value(final_date, price_data)
    voo_xirr = voo_portfolio.calculate_xirr(final_date, voo_final_value)
    
    print(f"VOO Benchmark Portfolio:")
    print(f"  Final Value: ${voo_final_value:,.2f}")
    print(f"  Total VOO shares: {voo_portfolio.holdings.get('VOO', 0):.4f}")
    if voo_xirr:
        print(f"  XIRR (Annualized Return): {voo_xirr:.2f}%")
else:
    print("VOO price data not available")

## Step 5: Fetch Price Data

In [ ]:
def compare_portfolios(actual_xirr, voo_xirr, actual_value, voo_value, transactions_df):
    """Compare actual vs VOO benchmark returns using Polars DataFrame"""
    
    # Calculate total invested using Polars
    total_invested = abs(
        transactions_df
        .filter(pl.col('Type') == 'BUY')
        .select(pl.col('Amount USD').sum())
        .item()
    )
    
    # Calculate time period using Polars
    date_range = transactions_df.select([
        pl.col('Trade Date').min().alias('start_date'),
        pl.col('Trade Date').max().alias('end_date')
    ]).row(0)
    
    start_date = date_range[0]
    end_date = date_range[1]
    years = (end_date - start_date).days / 365.25
    
    print("="*60)
    print("PORTFOLIO COMPARISON RESULTS")
    print("="*60)
    print(f"\nInvestment Period: {start_date} to {end_date} ({years:.1f} years)")
    print(f"Total Invested: ${total_invested:,.2f}")
    
    print(f"\nActual Portfolio:")
    print(f"  Final Value: ${actual_value:,.2f}")
    print(f"  Total Return: {((actual_value/total_invested - 1) * 100):.2f}%")
    if actual_xirr:
        print(f"  XIRR (Annualized): {actual_xirr:.2f}%")
    
    print(f"\nVOO Benchmark:")
    print(f"  Final Value: ${voo_value:,.2f}")
    print(f"  Total Return: {((voo_value/total_invested - 1) * 100):.2f}%")
    if voo_xirr:
        print(f"  XIRR (Annualized): {voo_xirr:.2f}%")
    
    if actual_xirr and voo_xirr:
        print(f"\nPerformance Analysis:")
        outperformance = actual_xirr - voo_xirr
        if outperformance > 0:
            print(f"  ✅ Your portfolio OUTPERFORMED VOO by {outperformance:.2f}% annually")
        else:
            print(f"  ❌ Your portfolio UNDERPERFORMED VOO by {abs(outperformance):.2f}% annually")
        
        # Calculate dollar difference
        dollar_diff = actual_value - voo_value
        if dollar_diff > 0:
            print(f"  💰 You have ${dollar_diff:,.2f} more than the VOO strategy")
        else:
            print(f"  💸 You have ${abs(dollar_diff):,.2f} less than the VOO strategy")

# Run comparison
if 'voo_xirr' in locals():
    compare_portfolios(actual_xirr, voo_xirr, final_value, voo_final_value, equity_transactions)

## Step 6: Calculate Actual Portfolio Performance

In [ ]:
# Portfolio composition analysis using Polars
holdings_summary = (
    equity_transactions
    .group_by('Ticker')
    .agg(pl.col('Amount USD').sum().abs().alias('total_amount'))
    .sort('total_amount', descending=True)
)
top_holdings = holdings_summary.head(10)

# Convert to pandas for matplotlib (visualization still needs pandas)
top_holdings_pd = top_holdings.to_pandas()
equity_transactions_pd = equity_transactions.to_pandas()

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Top holdings by investment amount
ax1 = axes[0, 0]
ax1.barh(top_holdings_pd['Ticker'], top_holdings_pd['total_amount'], color='steelblue')
ax1.set_title('Top 10 Holdings by Total Investment', fontsize=14, fontweight='bold')
ax1.set_xlabel('Total Invested ($)')
ax1.set_ylabel('Ticker')
ax1.invert_yaxis()

# 2. Transaction types distribution
ax2 = axes[0, 1]
txn_types = equity_transactions['Type'].value_counts().to_pandas()
colors = plt.cm.Set3(range(len(txn_types)))
txn_types.plot(kind='pie', ax=ax2, autopct='%1.1f%%', colors=colors)
ax2.set_title('Transaction Types Distribution', fontsize=14, fontweight='bold')
ax2.set_ylabel('')

# 3. Investment timeline
ax3 = axes[1, 0]
monthly_buys = (
    equity_transactions
    .filter(pl.col('Type') == 'BUY')
    .with_columns(
        pl.col('Trade Date').dt.to_string('%Y-%m').alias('YearMonth')
    )
    .group_by('YearMonth')
    .agg(pl.col('Amount USD').sum().abs().alias('monthly_amount'))
    .sort('YearMonth')
    .to_pandas()
)
ax3.bar(range(len(monthly_buys)), monthly_buys['monthly_amount'], color='green', alpha=0.7)
ax3.set_title('Monthly Investment Pattern', fontsize=14, fontweight='bold')
ax3.set_xlabel('Month')
ax3.set_ylabel('Amount Invested ($)')
ax3.set_xticks(range(0, len(monthly_buys), max(1, len(monthly_buys)//10)))
ax3.set_xticklabels(monthly_buys['YearMonth'].iloc[::max(1, len(monthly_buys)//10)], rotation=45)

# 4. Cumulative investment over time
ax4 = axes[1, 1]
buys_only = (
    equity_transactions
    .filter(pl.col('Type') == 'BUY')
    .sort('Trade Date')
    .with_columns(
        pl.col('Amount USD').abs().cum_sum().alias('Cumulative')
    )
    .to_pandas()
)
ax4.plot(buys_only['Trade Date'], buys_only['Cumulative'], linewidth=2, color='navy')
ax4.fill_between(buys_only['Trade Date'], 0, buys_only['Cumulative'], alpha=0.3, color='navy')
ax4.set_title('Cumulative Investment Over Time', fontsize=14, fontweight='bold')
ax4.set_xlabel('Date')
ax4.set_ylabel('Cumulative Investment ($)')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 7: Simulate VOO Benchmark Portfolio

In [ ]:
# Analyze returns by year using Polars
def analyze_by_year(transactions_df):
    """Break down investment activity by year using Polars"""
    
    yearly_analysis = (
        transactions_df
        .with_columns(
            pl.col('Trade Date').dt.year().alias('Year')
        )
        .group_by('Year')
        .agg([
            # Calculate invested amount (BUY transactions)
            pl.col('Amount USD')
                .filter(pl.col('Type') == 'BUY')
                .sum().abs()
                .alias('Invested'),
            
            # Calculate sold amount (SELL transactions)
            pl.col('Amount USD')
                .filter(pl.col('Type') == 'SELL')
                .sum().abs()
                .alias('Sold'),
            
            # Calculate dividends
            pl.col('Amount USD')
                .filter(pl.col('Type').is_in(['DIVIDEND', 'REINVEST']))
                .sum().abs()
                .alias('Dividends'),
            
            # Count transactions
            pl.count().alias('Transactions')
        ])
        .with_columns(
            (-pl.col('Invested') + pl.col('Sold')).alias('Net Cash Flow')
        )
        .sort('Year')
    )
    
    return yearly_analysis

yearly_analysis = analyze_by_year(equity_transactions)
print("Investment Activity by Year (Polars Analysis):")
print(yearly_analysis)

# Convert to pandas for visualization
yearly_pandas = yearly_analysis.to_pandas()

# Visualize yearly activity
fig, ax = plt.subplots(figsize=(12, 6))
x = yearly_pandas['Year']
width = 0.35
ax.bar(x - width/2, yearly_pandas['Invested'], width, label='Invested', color='green', alpha=0.7)
ax.bar(x + width/2, yearly_pandas['Sold'], width, label='Sold', color='red', alpha=0.7)
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Amount ($)', fontsize=12)
ax.set_title('Annual Investment and Sales Activity', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

In [ ]:
## Summary: Key Polars Features Demonstrated

This notebook showcased Polars for investment portfolio analysis with these key features:

### 1. **Performance Benefits**
- Lazy evaluation for query optimization
- Parallel execution of operations
- Memory-efficient columnar storage
- Streaming support for large datasets

### 2. **Expression API**
- Chainable operations: `filter()`, `with_columns()`, `group_by()`, `agg()`
- Window functions with `over()`
- Conditional logic with `when().then().otherwise()`
- Date/time operations with `.dt` namespace

### 3. **Data Operations**
- **Selection**: `select()`, column subsetting
- **Filtering**: `filter()` with boolean expressions
- **Aggregation**: `group_by()` with multiple aggregation functions
- **Joins**: `join()` for combining DataFrames
- **Pivoting**: `pivot()` for reshaping data
- **Sorting**: `sort()` with multiple columns

### 4. **Polars vs Pandas Comparison**
| Feature | Polars | Pandas |
|---------|--------|--------|
| Speed | 5-10x faster for large datasets | Baseline |
| Memory | More efficient, columnar storage | Row-based |
| API | Consistent, predictable | More flexible but inconsistent |
| Lazy Evaluation | Yes, with query optimization | No |
| Type Safety | Strong typing | Weaker typing |
| Null Handling | Explicit null type | NaN/None mixing |

### 5. **Common Polars Patterns Used**
```python
# Chain operations
df.filter(...).with_columns(...).group_by(...).agg(...)

# Lazy evaluation
df.lazy().filter(...).collect()

# Window functions
pl.col("amount").cum_sum().over("ticker")

# Conditional columns
pl.when(condition).then(value1).otherwise(value2)

# Date operations
pl.col("date").dt.year()
```

### Next Steps
- Use `streaming=True` in `collect()` for very large datasets
- Explore Polars plugins for specialized operations
- Consider Polars for ETL pipelines and data processing workflows

## Polars Lazy Evaluation and Query Optimization

## Step 8: Compare Results

In [ ]:
def compare_portfolios(actual_xirr, voo_xirr, actual_value, voo_value, transactions_df):
    """Compare actual vs VOO benchmark returns"""
    
    # Calculate total invested
    total_invested = abs(transactions_df[transactions_df['Type'] == 'Buy']['Amount USD'].sum())
    
    # Calculate time period
    start_date = transactions_df['Trade Date'].min()
    end_date = transactions_df['Trade Date'].max()
    years = (end_date - start_date).days / 365.25
    
    print("="*60)
    print("PORTFOLIO COMPARISON RESULTS")
    print("="*60)
    print(f"\nInvestment Period: {start_date.date()} to {end_date.date()} ({years:.1f} years)")
    print(f"Total Invested: ${total_invested:,.2f}")
    
    print(f"\nActual Portfolio:")
    print(f"  Final Value: ${actual_value:,.2f}")
    print(f"  Total Return: {((actual_value/total_invested - 1) * 100):.2f}%")
    if actual_xirr:
        print(f"  XIRR (Annualized): {actual_xirr:.2f}%")
    
    print(f"\nVOO Benchmark:")
    print(f"  Final Value: ${voo_value:,.2f}")
    print(f"  Total Return: {((voo_value/total_invested - 1) * 100):.2f}%")
    if voo_xirr:
        print(f"  XIRR (Annualized): {voo_xirr:.2f}%")
    
    if actual_xirr and voo_xirr:
        print(f"\nPerformance Analysis:")
        outperformance = actual_xirr - voo_xirr
        if outperformance > 0:
            print(f"  ✅ Your portfolio OUTPERFORMED VOO by {outperformance:.2f}% annually")
        else:
            print(f"  ❌ Your portfolio UNDERPERFORMED VOO by {abs(outperformance):.2f}% annually")
        
        # Calculate dollar difference
        dollar_diff = actual_value - voo_value
        if dollar_diff > 0:
            print(f"  💰 You have ${dollar_diff:,.2f} more than the VOO strategy")
        else:
            print(f"  💸 You have ${abs(dollar_diff):,.2f} less than the VOO strategy")

# Run comparison
if 'voo_xirr' in locals():
    compare_portfolios(actual_xirr, voo_xirr, final_value, voo_final_value, equity_transactions)

## Step 9: Visualizations

In [ ]:
# Portfolio composition analysis
holdings_summary = equity_transactions.groupby('Ticker')['Amount USD'].sum().abs().sort_values(ascending=False)
top_holdings = holdings_summary.head(10)

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Top holdings by investment amount
ax1 = axes[0, 0]
top_holdings.plot(kind='barh', ax=ax1, color='steelblue')
ax1.set_title('Top 10 Holdings by Total Investment', fontsize=14, fontweight='bold')
ax1.set_xlabel('Total Invested ($)')
ax1.set_ylabel('Ticker')

# 2. Transaction types distribution
ax2 = axes[0, 1]
txn_types = equity_transactions['Type'].value_counts()
colors = plt.cm.Set3(range(len(txn_types)))
txn_types.plot(kind='pie', ax=ax2, autopct='%1.1f%%', colors=colors)
ax2.set_title('Transaction Types Distribution', fontsize=14, fontweight='bold')
ax2.set_ylabel('')

# 3. Investment timeline
ax3 = axes[1, 0]
monthly_buys = equity_transactions[equity_transactions['Type'] == 'Buy'].copy()
monthly_buys['YearMonth'] = monthly_buys['Trade Date'].dt.to_period('M')
monthly_investment = monthly_buys.groupby('YearMonth')['Amount USD'].sum().abs()
monthly_investment.plot(kind='bar', ax=ax3, color='green', alpha=0.7)
ax3.set_title('Monthly Investment Pattern', fontsize=14, fontweight='bold')
ax3.set_xlabel('Month')
ax3.set_ylabel('Amount Invested ($)')
ax3.tick_params(axis='x', rotation=45)

# 4. Cumulative investment over time
ax4 = axes[1, 1]
buys_only = equity_transactions[equity_transactions['Type'] == 'Buy'].copy()
buys_only = buys_only.sort_values('Trade Date')
buys_only['Cumulative'] = buys_only['Amount USD'].abs().cumsum()
ax4.plot(buys_only['Trade Date'], buys_only['Cumulative'], linewidth=2, color='navy')
ax4.fill_between(buys_only['Trade Date'], 0, buys_only['Cumulative'], alpha=0.3, color='navy')
ax4.set_title('Cumulative Investment Over Time', fontsize=14, fontweight='bold')
ax4.set_xlabel('Date')
ax4.set_ylabel('Cumulative Investment ($)')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 10: Detailed Analysis by Year

In [ ]:
# Analyze returns by year
def analyze_by_year(transactions_df):
    """Break down investment activity by year"""
    df = transactions_df.copy()
    df['Year'] = df['Trade Date'].dt.year
    
    yearly_summary = []
    for year in sorted(df['Year'].unique()):
        year_data = df[df['Year'] == year]
        buys = year_data[year_data['Type'] == 'Buy']['Amount USD'].sum()
        sells = year_data[year_data['Type'] == 'Sell']['Amount USD'].sum()
        dividends = year_data[year_data['Type'].isin(['Dividend', 'Reinvest'])]['Amount USD'].sum()
        
        yearly_summary.append({
            'Year': year,
            'Invested': abs(buys),
            'Sold': abs(sells),
            'Dividends': abs(dividends),
            'Net Cash Flow': -abs(buys) + abs(sells),
            'Transactions': len(year_data)
        })
    
    yearly_df = pd.DataFrame(yearly_summary)
    return yearly_df

yearly_analysis = analyze_by_year(equity_transactions)
print("Investment Activity by Year:")
print(yearly_analysis.to_string(index=False))

# Visualize yearly activity
fig, ax = plt.subplots(figsize=(12, 6))
x = yearly_analysis['Year']
width = 0.35
ax.bar(x - width/2, yearly_analysis['Invested'], width, label='Invested', color='green', alpha=0.7)
ax.bar(x + width/2, yearly_analysis['Sold'], width, label='Sold', color='red', alpha=0.7)
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Amount ($)', fontsize=12)
ax.set_title('Annual Investment and Sales Activity', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## Conclusions and Next Steps

This notebook provides a comprehensive analysis of your investment portfolio performance using XIRR (money-weighted returns) compared to a VOO benchmark strategy.

### Key Insights:
1. **XIRR vs CAGR**: We use XIRR because it accounts for the timing and size of your cash flows, giving a more accurate picture of your investment performance
2. **Dividend Reinvestment**: Properly handled by tracking share increases without affecting XIRR cash flows
3. **Fair Comparison**: The VOO benchmark simulates investing the same amounts on the same dates as your actual investments

### To Run This Analysis:
1. Set your Nasdaq Data Link API key: `export NASDAQ_DATA_LINK_API_KEY="your_key"`
2. Run each cell sequentially to see results
3. The analysis will work with dummy data if no API key is provided (for testing)

### Potential Enhancements:
- Add support for stock splits and corporate actions
- Include transaction costs in the analysis
- Compare against multiple benchmarks (QQQ, IWM, etc.)
- Add risk-adjusted return metrics (Sharpe ratio, etc.)
- Generate a detailed PDF report of findings